# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 9:47AM,258308,15,VT80184569,"Virtus Pharmaceuticals, LLC",Released
1,Feb 28 2023 9:47AM,258308,15,VT80184570,"Virtus Pharmaceuticals, LLC",Released
2,Feb 28 2023 9:47AM,258308,15,VT80184571,"Virtus Pharmaceuticals, LLC",Released
3,Feb 28 2023 9:47AM,258308,15,VT80184572,"Virtus Pharmaceuticals, LLC",Released
4,Feb 28 2023 9:47AM,258308,15,VT80184573,"Virtus Pharmaceuticals, LLC",Released
5,Feb 28 2023 9:47AM,258308,15,VT80184574,"Virtus Pharmaceuticals, LLC",Released
6,Feb 28 2023 9:47AM,258308,15,VT80184581,"Virtus Pharmaceuticals, LLC",Released
7,Feb 28 2023 9:47AM,258308,15,VT80184582,"Virtus Pharmaceuticals, LLC",Released
8,Feb 28 2023 9:47AM,258308,15,VT80184583,"Virtus Pharmaceuticals, LLC",Released
9,Feb 28 2023 9:47AM,258308,15,VT80184584,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,258341,Released,1
45,258342,Released,1
46,258343,Released,1
47,258344,Released,1
48,258345,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258341,NaN,NaN,1.0
258342,NaN,NaN,1.0
258343,NaN,NaN,1.0
258344,NaN,NaN,1.0
258345,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258202,98.0,0.0,15.0
258219,0.0,0.0,61.0
258225,0.0,0.0,7.0
258227,0.0,0.0,1.0
258234,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258202,98,0,15
258219,0,0,61
258225,0,0,7
258227,0,0,1
258234,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258202,98,0,15
1,258219,0,0,61
2,258225,0,0,7
3,258227,0,0,1
4,258234,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258202,98,,15
1,258219,,,61
2,258225,,,7
3,258227,,,1
4,258234,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC"
16,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc
18,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated"
19,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc."
51,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health
52,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC"
73,Feb 28 2023 9:31AM,258251,15,"Brookfield Pharmaceuticals, LLC"
93,Feb 28 2023 9:30AM,258225,15,"Person & Covey, Inc."
100,Feb 28 2023 9:17AM,258342,12,Hush Hush
101,Feb 28 2023 9:16AM,258341,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",,,16
1,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,,,2
2,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",,,1
3,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",,,32
4,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,,,1
5,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC",,,21
6,Feb 28 2023 9:31AM,258251,15,"Brookfield Pharmaceuticals, LLC",,,20
7,Feb 28 2023 9:30AM,258225,15,"Person & Covey, Inc.",,,7
8,Feb 28 2023 9:17AM,258342,12,Hush Hush,,,1
9,Feb 28 2023 9:16AM,258341,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16,,
1,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,2,,
2,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",1,,
3,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",32,,
4,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,1,,
5,Feb 28 2023 9:33AM,258260,15,"Brookfield Pharmaceuticals, LLC",21,,
6,Feb 28 2023 9:31AM,258251,15,"Brookfield Pharmaceuticals, LLC",20,,
7,Feb 28 2023 9:30AM,258225,15,"Person & Covey, Inc.",7,,
8,Feb 28 2023 9:17AM,258342,12,Hush Hush,1,,
9,Feb 28 2023 9:16AM,258341,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16,,
1,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,2,,
2,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",1,,
3,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",32,,
4,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16.0,NaN,NaN
1,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,2.0,NaN,NaN
2,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",1.0,NaN,NaN
3,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",32.0,NaN,NaN
4,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 9:47AM,258308,15,"Virtus Pharmaceuticals, LLC",16.0,0.0,0.0
1,Feb 28 2023 9:41AM,258345,10,Eye Pharma Inc,2.0,0.0,0.0
2,Feb 28 2023 9:40AM,258344,10,"Senseonics, Incorporated",1.0,0.0,0.0
3,Feb 28 2023 9:39AM,258267,15,"Person & Covey, Inc.",32.0,0.0,0.0
4,Feb 28 2023 9:39AM,258343,20,Granules Consumer Health,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4132826,307.0,7.0,0.0
12,1291584,27.0,2.0,0.0
15,1549513,111.0,0.0,98.0
16,258329,0.0,1.0,0.0
19,2066586,19.0,14.0,0.0
20,516627,2.0,4.0,18.0
21,516558,2.0,0.0,0.0
22,258338,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4132826,307.0,7.0,0.0
1,12,1291584,27.0,2.0,0.0
2,15,1549513,111.0,0.0,98.0
3,16,258329,0.0,1.0,0.0
4,19,2066586,19.0,14.0,0.0
5,20,516627,2.0,4.0,18.0
6,21,516558,2.0,0.0,0.0
7,22,258338,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,307.0,7.0,0.0
1,12,27.0,2.0,0.0
2,15,111.0,0.0,98.0
3,16,0.0,1.0,0.0
4,19,19.0,14.0,0.0
5,20,2.0,4.0,18.0
6,21,2.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,307.0
1,12,Released,27.0
2,15,Released,111.0
3,16,Released,0.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
Executing,7.0,2.0,0.0,1.0,14.0,4.0,0.0,0.0
Released,307.0,27.0,111.0,0.0,19.0,2.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
1,Executing,7.0,2.0,0.0,1.0,14.0,4.0,0.0,0.0
2,Released,307.0,27.0,111.0,0.0,19.0,2.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,98.0,0.0,0.0,18.0,0.0,0.0
1,Executing,7.0,2.0,0.0,1.0,14.0,4.0,0.0,0.0
2,Released,307.0,27.0,111.0,0.0,19.0,2.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()